# Laboratory Session 3 -- Databases 

On this laboratory session, you will be learning how to use databases and why they are very useful for data science or for supplying data for machine learning projects. The session will be divided into two parts:

1. **Relational databases:** you will install PGAdmin and Postgres and learn the basics of seting up a database, querying in SQL, and connecting your database to python to query directly. You will use the [mimic-iii database](https://github.com/MIT-LCP/mimic-code), a critical care database.
2. **Graph databases:** you will be introduced to a different type of databases. This will be done through Neo4j Sandbox. You will not have to install any software, as for this example you will work online. After querying the tool using the Cypher language, you will also try to query your Neo4j database through Python. You will be using the movies database, containing a series of movies, and its actors and directors.

## Databases - MIMIC III
MIMIC-III is a large, freely-available database comprising deidentified health-related data associated with over forty thousand patients (**Note**: in this tutorial just a fraction will be used) who stayed in critical care units of the Beth Israel Deaconess Medical Center between 2001 and 2012. The database includes information such as demographics, vital sign measurements made at the bedside (~1 data point per hour), laboratory test results, procedures, medications, caregiver notes, imaging reports, and mortality (including post-hospital discharge).

MIMIC supports a diverse range of analytic studies spanning epidemiology, clinical decision-rule improvement, and electronic tool development. It is notable for three factors: it is freely available to researchers worldwide; it encompasses a diverse and very large population of ICU patients; and it contains highly granular data, including vital signs, laboratory results, and medications.

### Data description

MIMIC-III is a relational database consisting of 26 tables. Tables are linked by identifiers which usually have the suffix ‘ID’. For example, SUBJECT_ID refers to a unique patient, HADM_ID refers to a unique admission to the hospital, and ICUSTAY_ID refers to a unique admission to an intensive care unit.

Charted events such as notes, laboratory tests, and fluid balance are stored in a series of ‘events’ tables. For example the OUTPUTEVENTS table contains all measurements related to output for a given patient, while the LABEVENTS table contains laboratory test results for a patient.

Tables prefixed with ‘D_’ are dictionary tables and provide definitions for identifiers. For example, every row of CHARTEVENTS is associated with a single ITEMID which represents the concept measured, but it does not contain the actual name of the measurement. By joining CHARTEVENTS and D_ITEMS on ITEMID, it is possible to identify the concept represented by a given ITEMID

Tables composing the database:
- admissions: Hospital admissions associated with an ICU stay
- callout: Record of when patients were ready for discharge (called out), and the actual time of their discharge (or more generally, their outcome)
- caregivers: List of caregivers associated with an ICU stay
- chartevents: Events occuring on a patient chart
- cptevents: Events recorded in Current Procedural Terminology
- d_cpt: High-level dictionary of the Current Procedural Terminology
- d_icd_diagnoses: Dictionary of the International Classification of Diseases, 9th Revision (Diagnoses)
- d_icd_procedures: Dictionary of the International Classification of Diseases, 9th Revision (Procedures)
- d_items: Dictionary of non-laboratory-related charted items
- d_labitems: Dictionary of laboratory-related items
- datetimeevents: Events relating to a datetime
- diagnoses_icd: Diagnoses relating to a hospital admission coded using the ICD9 system
- drgcodes: Hospital stays classified using the Diagnosis-Related Group system
- icustays: List of ICU admissions
- inputevents_cv: Events relating to fluid input for patients whose data was originally stored in the CareVue database
- inputevents_mv: Events relating to fluid input for patients whose data was originally stored in the MetaVision database
- labevents: Events relating to laboratory tests
- microbiologyevents: Events relating to microbiology tests
- noteevents: Notes associated with hospital stays
- outputevents: Outputs recorded during the ICU stay
- patients: Patients associated with an admission to the ICU
- procedureevents_mv: Procedure start and stop times recorded for MetaVision patients
- prescriptions: Medicines prescribed
- procedureevents_mv: Procedure start and stop times recorded for MetaVision patients
- procedures_icd: Procedures relating to a hospital admission coded using the ICD9 system
- services: Hospital services that patients were under during their hospital stay
- transfers: Location of patients during their hospital stay


### Create Database

Install [postgresql](https://www.postgresql.org/download/), a open source object-relational database system that has earned it a strong reputation for reliability, feature robustness, and performance.

During installation, confirm that you install `pgAdmin 4` which is a free open source graphical management tool for PostgreSQL and derivative relational databases (as in the figure below)

<img src="postgresql.png" alt="postgresql" width="400" height="500">


After instalation, follow the steps to create your database:

1. Open `pgAdmin 4'
2. Create your own `username` and `passwourd`
3. Create a new server
    - Right click in Servers -> Create -> Server
        <img src="create_server1.png" alt="postgresql" width="400" height="500">
    - Name your server `mimic`
        <img src="create_server2.png" alt="postgresql" width="400" height="500">
    - In Connection tab insert: 
        1. Host name/adress as localhost
        2. Maintenance database as postgres
        3. Previous created credentials (username and password)
        <img src="create_server3.png" alt="postgresql" width="400" height="500">
4. Create database
    - Right click on mimic -> Create -> Database
        <img src="create_database.png" alt="postgresql" width="400" height="500">
    - Name your database `mimic-iii`
        <img src="create_database2.png" alt="postgresql" width="400" height="500">

With your server and database created, now it's time to populate your database. Right click in your database `mimic-iii`and choose `Query Tool`

<img src="query_tool.png" alt="postgresql" width="400" height="500">

Confirm that you are in the desired server and database in the upper left corner of Query Tool
<img src="query_tool2.png" alt="postgresql" width="400" height="500">

Open and run the .sql files in the following order:

1. Run `postgres_create_tables_mimic.sql` file to generate the MIMIC-III tables

**Important Note**: you should save your .csv files in a folder called data on `C:\data\mimic-iii\`
2. Run `postgres_load_data_mimic.sql` file to populate your database through `.csv` files.
3. Run `postgres_add_indexes_mimic.sql` file to generate tables indexes
4. Run `postgres_add_constraints_mimic.sql` file to generate correspondence between tables

With your database creates, you can now start querying.


### Structured Query Language (SQL)

### Python Connection

Install `psycopg2` library to connect python to postgresql allowing query. **Tip**: use the environment previously created to pratical classes

Write the following code to access to database. **Note**: change `user` and `password` to your credentials

In [6]:
import pandas as pd
import psycopg2 as psql

conn = psql.connect(host='localhost',
                    database='mimic-iii',
                    user='postgres',
                    password='postgres',
                    port=5433)

#Create a cursor object using the cursor() method
cursor = conn.cursor()

#Execute a SQL function using the execute() method
cursor.execute("select version()")

# Fetch a single row using fetchone() method
data = cursor.fetchone()
print("Connection established to: ", data)

# Fetch all rows using fetchall() method
cursor.execute("SELECT * FROM admissions")
data = cursor.fetchall()

# Convert your data to dataframe
admission_table = pd.DataFrame(data)

#Closing the connection
conn.close()

admission_table.head()

Connection established to:  ('PostgreSQL 14.1, compiled by Visual C++ build 1914, 64-bit',)


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
0,12258,10006,142345,2164-10-23 21:09:00,2164-11-01 17:15:00,NaT,EMERGENCY,EMERGENCY ROOM ADMIT,HOME HEALTH CARE,Medicare,None,CATHOLIC,SEPARATED,BLACK/AFRICAN AMERICAN,2164-10-23 16:43:00,2164-10-23 23:00:00,SEPSIS,0,1
1,12263,10011,105331,2126-08-14 22:32:00,2126-08-28 18:59:00,2126-08-28 18:59:00,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,DEAD/EXPIRED,Private,None,CATHOLIC,SINGLE,UNKNOWN/NOT SPECIFIED,NaT,NaT,HEPATITIS B,1,1
2,12265,10013,165520,2125-10-04 23:36:00,2125-10-07 15:13:00,2125-10-07 15:13:00,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,DEAD/EXPIRED,Medicare,None,CATHOLIC,None,UNKNOWN/NOT SPECIFIED,NaT,NaT,SEPSIS,1,1
3,12269,10017,199207,2149-05-26 17:19:00,2149-06-03 18:42:00,NaT,EMERGENCY,EMERGENCY ROOM ADMIT,SNF,Medicare,None,CATHOLIC,DIVORCED,WHITE,2149-05-26 12:08:00,2149-05-26 19:45:00,HUMERAL FRACTURE,0,1
4,12270,10019,177759,2163-05-14 20:43:00,2163-05-15 12:00:00,2163-05-15 12:00:00,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,DEAD/EXPIRED,Medicare,None,CATHOLIC,DIVORCED,WHITE,NaT,NaT,ALCOHOLIC HEPATITIS,1,1


### Dashboard and Visualization

Install [PowerBI](https://powerbi.microsoft.com/pt-pt/downloads/) which is a unified, scalable platform for self-service and enterprise business intelligence (BI) that’s easy to use and helps you gain deeper data insight.

Install [psqlODBC](https://www.postgresql.org/ftp/odbc/versions/msi/) (version in file `psqlodbc_13_02_0000.zip`). Open Database Connectivity (ODBC) is an open standard Application Programming Interface (API) for accessing a database and `psqlODBC`is the extension for Postgresql databases.

Follow the steps to configure the access to your database

1. After installation search in your computer for`ODBC Data Sources` as in figure:

<img src="ODBC.png" alt="postgresql" width="400" height="500">

2. Open `ODBC Data Sources` and add a new data origin with PostgreSQL Unicode

<img src="ODBC_create.png" alt="postgresql" width="400" height="500">

3. Insert your credentials as in the figure:

<img src="ODBC_create2.png" alt="postgresql" width="400" height="500">

4. Click `Test` in order to verify your connection. If connection unsucessful, re-write your credentials.

<img src="ODBC_create3.png" alt="postgresql" width="400" height="500">

After configuration open `PowerBI` and follow the steps:

1. Access `Get Data/Obter Dados`

<img src="get_data.png" alt="postgresql" width="400" height="500">

2. Choose `ODBC` option

<img src="get_data2.png" alt="postgresql" width="400" height="500">

3. Connect to previously created access 

<img src="get_data3.png" alt="postgresql" width="400" height="500">

4. Insert your credentials

<img src="get_data4.png" alt="postgresql" width="400" height="500">

Now, you have access to your database and can choose the necessary tables to visualize data. **Note**: the tables you need to select are indicated in the next chapter.

<img src="get_data5.png" alt="postgresql" width="400" height="500">


### PowerBI tutorial

The objective of this tutorial is to create a dashboard that shows patients' information about laboratory measures during their stay in hospital.

An example of the dashboard is presented below and is available to use in files provided:

In [8]:
from IPython.display import IFrame

a='https://app.powerbi.com/reportEmbed?reportId=439ad7ce-73c8-4163-ad0c-162040adb0d5&autoAuth=true&ctid=0bfa8500-b1f2-4566-baf1-6f59370893e7&config=eyJjbHVzdGVyVXJsIjoiaHR0cHM6Ly93YWJpLW5vcnRoLWV1cm9wZS1yZWRpcmVjdC5hbmFseXNpcy53aW5kb3dzLm5ldC8ifQ%3D%3D'

In [11]:
IFrame(a,width=1000, height=600)

#### Step 1 - Select tables
After database connection, select the following tables:
- admissions
- d_labitems
- labevents
- patients

#### Step 2 - Create relationships between tables
<img src="bi_relation.png" alt="postgresql" width="400" height="500">

Create the following relations:
- d_labitems and labevents - One to many in `itemid` column
- labevents and admissions - Many to one in `hadm_id` column
- patients and admissions - One to many in `subject_id` column (this one probably will be done already)

<img src="bi_relation2.png" alt="postgresql" width="400" height="500">

#### Step 3 - Explore and create yout own dashboard

Try to recreate the dashboard above (or similar)

<img src="bi_charts.png" alt="postgresql" width="400" height="500">

Few tips to use power bi:
- In the blue box, you have all the tables and columns. Usually these are represented as `[table].[column]`. To use columns it is only necessary to drag to the workspace and then choose a visualization tool.
- In red, you have the visualization tools necessary to use for the dashboard (you are free to use the tools you desire)
     1. Line Graphs - plot `[labevents].[charttime]` vs `[labevents].[valuenum]` to visualize laboratory measures over time and `[labevents].[itemid]` as legend
     2. Filters - use `[patients].[subject_id]`, `[admissions].[hadm_id]` and `[labevents].[itemid]` to filter the data in all dashboard  
     3. Tables - reproduce the tables with the necessary columns
     3. Pie chart - represent information about `[patients].[gender]` and `[admissions].[admission_type]`
- In the green box, you have all the tools to format your graphs (tile,axis,font,...) including some analytical tools (max, min, mean,...)

